# Easy Tensorflow Lite
This Colab draws heavily from the Tensorflow Sine Model [example](https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/hello_world/train/train_hello_world_model.ipynb). For more information regarding what each component of this does, check out that example or look at the course listed in the README file on [GitHub](https://github.com/TSprech/Easy-Tensorflow-Lite).

## Setup
Imports and loading data

In [0]:
import os
MODELS_DIR = 'models/'
os.mkdir(MODELS_DIR) #Once the directory is set up, this can be commented out if run again
MODEL_TF = MODELS_DIR + 'model.pb'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

Minimum imports required to train model practically

In [2]:
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras

import pandas as pd
import numpy as np

2.2.0


## Dataset
Pull the dataset from GitHub which contain 1000 samples of number mapped to others, such that 0:1, 1:2, 2:0. There are also a few data points where this is not true, such that 1:1 or 0:2, to better illustrate that perfect data is not required.

In [0]:
dataset = pd.read_csv("https://raw.githubusercontent.com/TSprech/Easy-Tensorflow-Lite/develop/Sample_Data/Button_Data_1000_Samples.csv")

Convert the dataset from a Pandas data frame to a Numpy array

In [0]:
dataset = dataset.to_numpy()
dataset = dataset.astype('float32')

In [5]:
type(dataset) # Check to make sure it was converted

numpy.ndarray

In [6]:
SAMPLES = 1000

button_in_values = dataset[:,0] # Pull the first column for button in data
button_out_values = dataset[:,1] # Pull the second column for button out data
print(button_in_values)
print(button_out_values)

[0. 0. 1. ... 1. 0. 2.]
[1. 1. 2. ... 2. 1. 0.]


## Dataset Division
Break the data up into train, test, and validate data sets.

In [0]:
# 60% of the data for training, 20% for testing, 20% for validation
TRAIN_SPLIT =  int(0.6 * SAMPLES)
TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)

In [0]:
# Splitting the data into each set
button_in_train, button_in_test, button_in_validate = np.split(button_in_values, [TRAIN_SPLIT, TEST_SPLIT])
button_out_train, button_out_test, button_out_validate = np.split(button_out_values, [TRAIN_SPLIT, TEST_SPLIT])

## Building the Model
The sequential model is created and its layers added. As this is a simple model, a single dense input layer with 10 neurons and single output layer is all that is required to get an accurate prediction.

In [0]:
litemodel = tf.keras.Sequential()

litemodel.add(keras.layers.Dense(10, activation='relu', input_shape=(1,))) # First layer of neurons utilizing the ReLU function for activation

litemodel.add(keras.layers.Dense(1)) # Final layer is a single neuron, since we want to output a single value

litemodel.compile(optimizer='adam', loss='mse', metrics=['mae']) # Compile the model using a standard optimizer and loss function for regression

In [10]:
litemodel.summary() # This gives a basic summary to make sure our model looks as intended

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                20        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


## Training the Model
Here the training occurs, the dataset, epochs, and more are fed in to train the model. This generates a lot of data, and looking at the Sine Model linked at the start can show the different data and how to graph it in a meaningful sense. These graphs are excluded in this notebook as in the most technical sense they are unnecessary, especially for a model this simple. However, when developing more complex models they are borderline required.

In [11]:
hist = litemodel.fit(button_in_train, button_out_train, epochs=200, batch_size=16, validation_data=(button_in_validate, button_out_validate)) # Training the model

Epoch 1/200
38/38 [==============================] - 0s 8ms/step - loss: 1.3596 - mae: 1.0875 - val_loss: 1.2811 - val_mae: 1.0776
Epoch 2/200
38/38 [==============================] - 0s 2ms/step - loss: 1.2192 - mae: 1.0571 - val_loss: 1.1740 - val_mae: 1.0461
Epoch 3/200
38/38 [==============================] - 0s 2ms/step - loss: 1.1287 - mae: 1.0274 - val_loss: 1.0923 - val_mae: 1.0127
Epoch 4/200
38/38 [==============================] - 0s 2ms/step - loss: 1.0559 - mae: 0.9936 - val_loss: 1.0243 - val_mae: 0.9788
Epoch 5/200
38/38 [==============================] - 0s 2ms/step - loss: 0.9919 - mae: 0.9598 - val_loss: 0.9625 - val_mae: 0.9440
Epoch 6/200
38/38 [==============================] - 0s 2ms/step - loss: 0.9321 - mae: 0.9252 - val_loss: 0.9041 - val_mae: 0.9089
Epoch 7/200
38/38 [==============================] - 0s 2ms/step - loss: 0.8762 - mae: 0.8905 - val_loss: 0.8492 - val_mae: 0.8741
Epoch 8/200
38/38 [==============================] - 0s 2ms/step - loss: 0.8204 - m

## Testing the model
Next the model is check to make sure the input creates the expected output. This is formatted such that the first column is the input, the second column is the output number from the original dataset, and the third is the output predicted by the model.

In [12]:
button_out_predict = np.round(litemodel.predict(button_in_test))

print("Input : Output : Predict")
for i in range(len(button_out_predict)):
  print('%i     : %i      : %i' %(button_in_test[i], button_out_test[i], button_out_predict[i]))

Input : Output : Predict
0     : 1      : 1
0     : 1      : 1
1     : 2      : 2
2     : 0      : 0
0     : 1      : 1
2     : 0      : 0
1     : 2      : 2
0     : 1      : 1
1     : 1      : 2
0     : 1      : 1
1     : 2      : 2
0     : 1      : 1
2     : 0      : 0
1     : 2      : 2
0     : 1      : 1
2     : 1      : 0
0     : 1      : 1
2     : 0      : 0
1     : 2      : 2
0     : 1      : 1
1     : 2      : 2
0     : 1      : 1
2     : 0      : 0
1     : 2      : 2
0     : 1      : 1
0     : 1      : 1
0     : 1      : 1
1     : 2      : 2
2     : 0      : 0
0     : 1      : 1
2     : 0      : 0
1     : 2      : 2
0     : 1      : 1
2     : 2      : 0
0     : 1      : 1
1     : 2      : 2
0     : 1      : 1
2     : 0      : 0
1     : 2      : 2
0     : 1      : 1
2     : 0      : 0
0     : 1      : 1
2     : 0      : 0
1     : 2      : 2
0     : 1      : 1
1     : 2      : 2
0     : 1      : 1
2     : 0      : 0
1     : 2      : 2
0     : 1      : 1
1     : 2      : 2
0     

## Converting the model
The code below is all utilized to convert the model into a lite model and then to a C buffer. If a trained model on a micro controller returns an overflow value, but the model works in Colab, there is likely an error in the converter code.

In [13]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(litemodel)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
  for i in range(500):
    yield([button_in_train[i].reshape(1, 1)])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization (except inputs/outputs which are always float)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

1680

In [14]:
# Install xxd
!apt-get update && apt-get -qq install xxd
# Convert to a C source file
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [931 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 

## The Output C Buffer
Copy this data along with the length of the buffer into the C project file.

In [15]:
# Print the C source file
!cat {MODEL_TFLITE_MICRO}

unsigned char g_model[] = {
  0x10, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0xde, 0xfe, 0xff, 0xff, 0x03, 0x00, 0x00, 0x00,
  0x2c, 0x06, 0x00, 0x00, 0x0c, 0x01, 0x00, 0x00, 0xf4, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 0xe0, 0x00, 0x00, 0x00,
  0xcc, 0x00, 0x00, 0x00, 0x90, 0x00, 0x00, 0x00, 0x6c, 0x00, 0x00, 0x00,
  0x48, 0x00, 0x00, 0x00, 0x24, 0x00, 0x00, 0x00, 0x18, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x90, 0xfe, 0xff, 0xff, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0xa0, 0xfe, 0xff, 0xff,
  0x00, 0x00, 0x00, 0x00, 0xa6, 0xff, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00,
  0x0a, 0x00, 0x00, 0x00, 0xcb, 0x3a, 0x81, 0x27, 0x32, 0xc8, 0x8f, 0xff,
  0x29, 0xd6, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0xc6, 0xff, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x0a, 0x00, 0x00, 0x00,
  0xc9, 0x58, 0x65, 0xf2, 0x4d, 0xff, 0x7f, 0xc7, 0xec, 0xde, 0x00, 0x00,
  0x00, 0x